<a href="https://colab.research.google.com/github/DeKUT-DSAIL/DSA-2024-NLP/blob/main/pre-lab/simple_nlp_prelab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing (NLP)
## What is Natural Language Processing?
NLP is the study of how computers can interact with humans in natural human language, according to [[3]](https://www.oracle.com/ke/artificial-intelligence/what-is-natural-language-processing/).

## Introduction
In this lesson, we shall consider some foundational concepts in Natural Language Processing like stemming, lemmatization and vectorization of text (converting text data into vectors).

## Objectives
* Distinguish between stemming and lemmatization.
* Describe stopwords and why they are removed.
* Describe tokenization in the context of NLP.
* Understand various vectorization techniques.
* Use probability to predict the next word in a sentence.

# Basic NLP Lab
In this section, we shall practice some basic NLP tasks, below are some of the terms that will be used in the lab:

## Corpus
A corpus is a structured set of texts that is machine-readable and sampled to be representative of a natural language. It is mostly used for linguistic research and analysis [4].

## Document
In NLP a document is a single unit of text that can be part of a corpus. It could be a tweet, an email, a report, an article, a book, etc.

## Creating a Bag of words
When working with text data, it is recommended that one vectorizes the text by creating a bag of words. Bag of words (BoW) is a way of representing text as an unordered collection that disregards grammar but retains the mutiplicity of the words i.e the number of times a word occurs in a text. This can be achieved through tokenization.

## Basic Cleaning and Tokenization
Using the sentence : "Apple shareholders have had a great year. Apple's stock price has gone steadily upwards -- Apple even broke a trillion-dollar valuation, continuing the dominance of this tech stock." as our example.
Consider capitalization, punctuation. e.g a computer would consider Apple and Apple's as different.

## Stemming Vs. Lemmatization and Stopwords
For smaller datasets, stemming, lemmatization and stopwords removal could be helpful in reducing the dimensionality of the data.
Stemming and lemmatization are text preprocessing techniques that are used in NLP to reduce the inflected forms of words across across a text data set to one common root word. For example, the word sing can have affixated forms like sang, sung, singing, song, songs and singer. Another word dance, can have dancing, danced, dances and dancer.

### Stemming
Stemmming is the elimination of suffixes in words to return it to its root form. To illustrate, we know that adding and 's' to the end of a word makes it plural, if a stemming algorithm was given the word "dogs", it would return "dog". Stems do not have to make actual English sense. For example, "historical" would be reduced to "histori" and not "history".

### Lemmatization
Lemmatization differs from stemming in that it considers the morphology of the word (structure) and attempts to reduce each word into its most basic form (lemma) that can be found in the English dictionary. For example, "historical" would be reduced to "history".

## Stopwords
These are words that contain little or no actual information. Exmples include words like:
* to
* the
* of

In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 12.6 MB/s eta 0:00:00


In [ ]:
# Load dependencies
import numpy as np
import re
import requests
import nltk
nltk.download('gutenberg', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
from nltk.corpus import gutenberg, stopwords, wordnet
from nltk import FreqDist, word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.util import trigrams
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import Counter
import spacy
from spacy.tokenizer import Tokenizer
import string
import pathlib
import math
import contractions
from itertools import islice

In the code cells below, we are going to use **The Great Gatsby** by **F. Scott Fitzgerald**, **The Adventures of Sherlock Holmes** by **Athur Conan Doyle** and **Adventures of Huckleberry Finn** by **Mark Twain**. These books are freely available at [Project Gutenberg](https://www.gutenberg.org/), we have scraped them from the internet .

Demonstrations shall be done using the text from The Great Gatsby and the practice shall be done using the text from Adventures Of Huckleberry Finn or The Adventures of Sherlock Holmes.

The libraries that shall be used heavily in this lab are natural language toolkit ([nltk](https://www.nltk.org/)), reguar expression operations ([re](https://docs.python.org/3/library/re.html)) and [spacy](https://spacy.io/). In order to use spacy more effectively, kindly use a GPU.

In [ ]:
def request_gutenberg(url):

  # Make a request to the url
  response = requests.get(url)

  # check if the request was successful
  if response.status_code == 200:
    book = response.text

  else:
    print(f"Failed to retrieve the text version. Status code: {response.status_code}")

  # remove unwanted new line and tab characters from the text, replacing with whitespace
  unwanted_chars = ["\n", "\r", "\d", "\t"]
  for char in unwanted_chars:
      book = book.replace(char, " ")

  return book

# URLs of the books to be used in this lab
g_gatsby_url = "https://www.gutenberg.org/cache/epub/64317/pg64317.txt"
huckleberry_finn_url = "https://www.gutenberg.org/cache/epub/76/pg76.txt"
sherlock_holmes_url = "https://www.gutenberg.org/files/1661/1661-0.txt"

# Make requests for each book
g_gatsby = request_gutenberg(g_gatsby_url)[1494:277912] # Remove introduction and footnotes
h_berry = request_gutenberg(huckleberry_finn_url)[9528:-18862]
s_holmes = request_gutenberg(sherlock_holmes_url)[1508:-18859]

books = {
    "The Great Gatsby": g_gatsby,
    "Adventures of Huckleberry Finn": h_berry,
    "The Adventures of Sherlock Holmes": s_holmes
    }

# Preview
for key, val in books.items():
  print(f"{key}:")
  print(val[:500])
  print("="*100)

The Great Gatsby:
I    In my younger and more vulnerable years my father gave me some advice  that I’ve been turning over in my mind ever since.    “Whenever you feel like criticizing anyone,” he told me, “just  remember that all the people in this world haven’t had the advantages  that you’ve had.”    He didn’t say any more, but we’ve always been unusually communicative  in a reserved way, and I understood that he meant a great deal more  than that. In consequence, I’m inclined to reserve all judgements, a  habi
Adventures of Huckleberry Finn:
CHAPTER I.      You don’t know about me without you have read a book by the name of The  Adventures of Tom Sawyer; but that ain’t no matter. That book was made  by Mr. Mark Twain, and he told the truth, mainly. There was things  which he stretched, but mainly he told the truth. That is nothing. I  never seen anybody but lied one time or another, without it was Aunt  Polly, or the widow, or maybe Mary. Aunt Polly—Tom’s Aunt Polly, she  is—and Mar

In [ ]:
def fix_encoding(text):
    # Encode the incorrectly decoded text back to bytes
    bytes_text = text.encode('latin1')
    # Decode the bytes using the correct encoding (UTF-8)
    corrected_text = bytes_text.decode('utf-8')
    return corrected_text

# Fix encoding of sherlock holmes
s_holmes = fix_encoding(s_holmes)
s_holmes

## Frequency Distributions
The  frequency distributions of words in a document can be extracted to know which words appear more\less frequently in a document. But in order to achieve this, the text must first be tokenized. Tokenization is basically breaking down the document into smaller pieces known as tokens which can be words or characters. For example, "Hello, world!" might result in the tokens ["Hello", ",", "" , "world", "!".].

In order to consider the value of the information captured in the tokens and also reduce the dimensionality of the data, one might consider to lemmatize (lemmatization is preferred because it considers the morphology in the text), remove stopwords and punctuation in the text.

Depending on the source of the text and the use case, one might consider removing links and non-word characters like emoticons.

**N/B**: Cleaning of the text such as removal of stopwords and numbers or even conversion of numbers in digit form to word form ought to be thought through carefully, after carefully considering the task at hand.

To get familiar with regex pattern, we recommend that you could experiment with [regex 101](https://regex101.com/) and consider some cheatsheets like [datacamp](https://images.datacamp.com/image/upload/v1665049611/Marketing/Blog/Regular_Expressions_Cheat_Sheet.pdf) and [dataquest](https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf) to start with.

In [ ]:
def tokenize_text(text):
    pattern = r"[a-zA-Z]+(?:'[a-z]+)?"  # Regex pattern to match words, including those with apostrophes
    text_tokens = nltk.regexp_tokenize(text, pattern)  # Tokenize using the regex pattern
    lemmatizer = WordNetLemmatizer()
    text_tokens = [lemmatizer.lemmatize(word.lower().strip()) for word in text_tokens]  # Convert tokens to lowercase, lemmatize, remove extra whitespace
    return text_tokens

g_gatsby_tokens = tokenize_text(g_gatsby) # tokenize the text
g_gatsby_freqdist =FreqDist(g_gatsby_tokens) # Get frequency distributions
g_gatsby_freqdist.most_common(10)

[('the', 2397),
 ('a', 1684),
 ('and', 1568),
 ('i', 1391),
 ('to', 1130),
 ('of', 1119),
 ('he', 854),
 ('in', 808),
 ('wa', 768),
 ('it', 646)]

Notice how a good number of the 10 most common words are words like 'the', 'and', 'of', and 'to'. These are stopwords which do not contain any meaningful information and they can be removed to reduce the dimensionality of the data.

The function to tokenize text has already been made for you, practice using the text from huckleberry finn.

In [ ]:
# Using Adventures of Huckelberry Finn / The Adventures of Sherlock Holmes
# Your code here


## Removing stopwords and punctuation
 Removal of stopwords helps in the reducing the dimensionality of the data. Words like 'the', 'and', 'of' are considered as stopwords because they contain little to no information, thus they are removed.

 In the code cells below:
 1. we shall create a list of stopwords from nltk's stopwords, then get the frequency distribution of the words in the text having already removed stopwords. What are the top 10 most common tokens in the Great Gatsby?
 2. We shall create a function called clean text to clean the Great Gatsby text. This function should deal with contractions like "don't", lowercase the text, remove non-word characters taking into account possessives, remove stopwords and extra whitespace, and lemmatize the text (not necessarily in this order), but instead of returning it as tokens, it should be returned as text.



In [ ]:
stopwords_list = stopwords.words('english') # List of stopwords

g_gatsby_stopped = [word for word in g_gatsby_tokens if word not in stopwords_list] # Exclude stopwords and punctuation
g_gatsby_stopped_freq = FreqDist(g_gatsby_stopped) # Get frequency distribution
g_gatsby_stopped_freq.most_common(10)

[('wa', 768),
 ('gatsby', 263),
 ('said', 235),
 ('tom', 191),
 ('daisy', 186),
 ('one', 155),
 ('like', 122),
 ('mr', 115),
 ('man', 114),
 ('back', 109)]

In [ ]:
# Using Adventures of Huckleberry Finn/The Adventures of Sherlock Holmes

# Your code here

In [ ]:
# Using spacy for nlp
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Using spacy
def clean_text1(text):
  text = contractions.fix(text) # Fix contractions
  text = text.lower() # lowercase the text
  text = text.replace('\n', ' ') # replace new line characters with whitespace
  text = re.sub(r"[^\w\s']", ' ', text) # Remove non-word characters, except apostroophes - to handle possessives correctly e.g "John's"
  text = text.strip() # Remove extra whitespace

  ## -- Tokenize, lemmatize, remove stopwords -- ##
  doc = nlp(text)
  tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct] # Lemmatize and exclude stopwords
  text = " ".join(tokens)
  # Remove any additional extra whitespace
  text = re.sub(r'\s+', ' ', text).strip()

  return text

In [ ]:
# Function to map NLTK Parts of Speech (POS) tags to WordNet POS tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [ ]:
# Using nltk
def clean_text2(text):
  # Fix contractions
  text = contractions.fix(text)
  text = text.lower() # lowercase
  text = text.replace('\n', ' ') # replace new line characters

  # Remove non-word chaaraacters except apostrophe's and handle possessives
  text = re.sub(r"[^\w\s']", ' ', text)
  text = text.strip() # Remove extra whitespace

  # Tokenize using regex pattern that captures words and common contractions/possessives
  pattern = r"[a-zA-Z]+(?:'[a-z]+)?"
  stopwords_list = set(stopwords.words('english'))
  text_tokens = nltk.regexp_tokenize(text, pattern)  # Tokenize using the regex pattern
  text_tokens = [word for word in text_tokens if word not in stopwords_list]

  # Tag parts of speech
  pos_tags = nltk.pos_tag(text_tokens)

  # Lemmatize based on POS tags
  lemmatizer = WordNetLemmatizer() # Init Lemmatizer
  lemmatized_tokens = []
  for word, tag in pos_tags:
    wn_tag = get_wordnet_pos(tag)
    if wn_tag:
      lemmatized_tokens.append(lemmatizer.lemmatize(word, pos=wn_tag))
    else:
      lemmatized_tokens.append(lemmatizer.lemmatize(word))

  cleaned_text = " ".join(lemmatized_tokens)
  cleaned_text = re.sub(r"\s+", ' ', cleaned_text).strip() # Remove any extra whitespace

  return cleaned_text


In [ ]:
# Clean the great gatbsy using the first function
g_gatsby_cleaned1 = clean_text1(g_gatsby)
print(g_gatsby_cleaned1[:200])

young vulnerable year father give advice turn mind feel like criticize tell remember people world advantage unusually communicative reserve way understand mean great deal consequence inclined reserve 


In [ ]:
# clean the great gatsby using the second function
g_gatsby_cleaned2 = clean_text2(g_gatsby)
print(g_gatsby_cleaned2[:200])

young vulnerable year father give advice turning mind ever since whenever feel like criticize anyone tell remember people world advantage say always unusually communicative reserve way understood mean


In [ ]:
# Using either/both functions clean the text of Adentures of Huckleberry Finn/The Adventures of Sherlock Holmes

# Your code here


## N-grams

N-grams are continuous sequences of n words from a given text. Types of n-grams include:
* Unigrams - 1 word e.g ('God'). This can also be viewed as frequency distribution.
* Bigrams - 2 words e.g ('Lord', 'God')
* Trigrams - 3 words e.g ('Lord', 'God', 'said')

In the code cell below, find the bigrams and trigams of the tokenized text (which excludes stopwords) that you created above.

What are the top 10 for both bigram and trigram collections?

Hint: Nltk's collocations provides an easy way to find these bigrams and trigrams. You could also use your own custom code

In [ ]:
## Using nltk's collocations
# Bigrams
bigram_measures = nltk.collocations.BigramAssocMeasures()
g_gatsby_bigram_finder = BigramCollocationFinder.from_words(g_gatsby_stopped)
g_gatsby_bigrams_scored = g_gatsby_bigram_finder.score_ngrams(bigram_measures.raw_freq)

# Trigrams
trigram_measures = nltk.collocations.TrigramAssocMeasures()
g_gatsby_trigram_finder = TrigramCollocationFinder.from_words(g_gatsby_stopped)
g_gatsby_trigrams_scored = g_gatsby_trigram_finder.score_ngrams(trigram_measures.raw_freq)
g_gatsby_trigrams_scored[:50]

[(('old', 'sport', 'said'), 0.0002464268112370626),
 (('said', 'mr', 'wolfshiem'), 0.0002464268112370626),
 (('doctor', 'j', 'eckleburg'), 0.0002053556760308855),
 (('new', 'york', 'wa'), 0.0002053556760308855),
 (('west', 'egg', 'village'), 0.0002053556760308855),
 (('eye', 'doctor', 'j'), 0.0001642845408247084),
 (('look', 'old', 'sport'), 0.0001642845408247084),
 (('oh', 'ga', 'od'), 0.0001642845408247084),
 (('copy', 'town', 'tattle'), 0.0001232134056185313),
 (('demanded', 'tom', 'suddenly'), 0.0001232134056185313),
 (('first', 'time', 'saw'), 0.0001232134056185313),
 (('ga', 'od', 'oh'), 0.0001232134056185313),
 (('long', 'time', 'ago'), 0.0001232134056185313),
 (('longest', 'day', 'year'), 0.0001232134056185313),
 (('od', 'oh', 'ga'), 0.0001232134056185313),
 (('two', 'young', 'woman'), 0.0001232134056185313),
 (('absolutely', 'real', 'page'), 8.21422704123542e-05),
 (('ah', 'h', 'h'), 8.21422704123542e-05),
 (('always', 'watch', 'longest'), 8.21422704123542e-05),
 (('another', 

In [ ]:
# Using custom code - feel free to make your own version of such code
def ngrams(tokens, n):
  return zip(*[tokens[i:] for i in range(n)])

def ngram_freq(text, n):
  doc = nlp(text)
  tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
  # Generate ngrams
  ngrams_list = list(ngrams(tokens, n))

  # Count frequencies
  ngrams_freq = Counter(ngrams_list)
  return ngrams_freq.most_common()

# get the ngram frequencies using custom code
gatsby_trigrams = ngram_freq(g_gatsby_cleaned1, 3)
gatsby_bigrams = ngram_freq(g_gatsby_cleaned1, 2)

# Preview
gatsby_trigrams[:50]

[(('gatsby', 's', 'house'), 11),
 (('come', 'gatsby', 's'), 7),
 (('west', 'egg', 'village'), 5),
 (('doctor', 't', 'j'), 5),
 (('t', 'j', 'eckleburg'), 5),
 (('eye', 'doctor', 't'), 4),
 (('look', 'old', 'sport'), 4),
 (('world', 's', 'series'), 4),
 (('gatsby', 's', 'face'), 4),
 (('wilson', 's', 'body'), 4),
 (('oh', 'ga', 'od'), 4),
 (('daisy', 's', 'house'), 4),
 (('live', 'west', 'egg'), 3),
 (('long', 'day', 'year'), 3),
 (('demand', 'tom', 'suddenly'), 3),
 (('copy', 'town', 'tattle'), 3),
 (('mrs', 'wilson', 's'), 3),
 (('beg', 'pardon', 'mr'), 3),
 (('s', 'house', 'summer'), 3),
 (('gatsby', 's', 'door'), 3),
 (('gatsby', 's', 'drive'), 3),
 (('old', 'sport', 'gatsby'), 3),
 (('daisy', 's', 'face'), 3),
 (('daisy', 's', 'voice'), 3),
 (('gatsby', 's', 'party'), 3),
 (('long', 'time', 'ago'), 3),
 (('hot', 'hot', 'hot'), 3),
 (('gatsby', 's', 'eye'), 3),
 (('gatsby', 's', 'car'), 3),
 (('myrtle', 'wilson', 's'), 3),
 (('ga', 'od', 'oh'), 3),
 (('od', 'oh', 'ga'), 3),
 (('east'

In [ ]:
# Use nltk's collocations on Adventures of Huckleberry Finn/The Adventures of Sherlock Holmes to get ngrams

# Your code here


In [ ]:
# Use custom code on Adventures of Huckleberry Finn/The Adventures of Sherlock Holmes to get ngrams

# Your code here

## Pointwise Mutual Information (PMI)
Pointwise mutual information is a measure of association that compares the probability of two events ocurring together to what this probability would be if the events were independent [[5]](https://en.wikipedia.org/wiki/Pointwise_mutual_information).
PMI helps us find related words, in simpler terms. It explains the co-occurence of 2 words together than we would expect by chance, it asks the question, "does the occurrence of one word depend on the occurrence of another word?"

For example the word "Data Science" has a specific meaning when these two words "Data" and "Science" go together. Otherwise meaning of these two words are independent. Similarly "Great Britain" is meaningful since we know the word "Great" can be used with several other words but not so relevant in meaning like "Great UK, Great London, Great Dubai etc."


This can be expressed mathematically as:
\begin{equation}
PMI(w_1, w_2) = log_2 \frac{P(w_1, w_2)}{P(w_1)P(w_2)}
\end{equation}

where:
* $PMI(w_1,w_2)$ is the pointwise mutual information of word 1 and word2
* $P(w_1,w_2)$ is the probability of word 1 and word 2 occurring together. This is considered as an intersection, $P(w_1 ∩ w_2)$.
* $P(w_1)$ and $P(w_2)$ is the probability of word 1 and word 2 occurring individually.

The probability of a word occuring can be calculated as:
\begin{equation}
P(w) = \frac{N(w)}{T}
\end{equation}
where:
* $N(w)$ is the frequency of word, w.
* $T$ is the total word count.
* $P(w)$ is the probability of word, w occurring.

Find the PMI scores of the great_gatsby, use the tokens instead of the text.

Hint: Use [NLTK's BigramCollocationFinder](https://www.nltk.org/api/nltk.collocations.BigramCollocationFinder.html#:~:text=A%20tool%20for%20the%20finding,than%20constructing%20an%20instance%20directly.&text=Construct%20a%20BigramCollocationFinder%2C%20given%20FreqDists,possibly%20non%2Dcontiguous%20bigrams.) for an easier way out, and apply a frequency filter of 5.

One can also come up with their own function to calculate the PMI scores.


In [ ]:
# PMI for the Great Gatsby
gatsby_pmi_finder = BigramCollocationFinder.from_words(g_gatsby_stopped)
gatsby_pmi_finder.apply_freq_filter(5) # use s as the frequency filter, meaning words that appear together
gatsby_pmi_scored = gatsby_pmi_finder.score_ngrams(bigram_measures.pmi)
gatsby_pmi_scored[:50]

[(('beg', 'pardon'), 11.98655314975666),
 (('j', 'eckleburg'), 11.501126322586419),
 (('doctor', 'j'), 10.98655314975666),
 (('dan', 'cody'), 10.664625054869294),
 (('ash', 'heap'), 10.385649105166483),
 (('meyer', 'wolfshiem'), 9.349123229141368),
 (('miss', 'baker'), 8.756547544310335),
 (('west', 'egg'), 8.741792915391757),
 (('egg', 'village'), 8.44223263353285),
 (('new', 'york'), 8.400412457838224),
 (('long', 'island'), 8.032356839369784),
 (('old', 'sport'), 8.027195134254006),
 (('half', 'dozen'), 7.816628148314345),
 (('east', 'egg'), 7.72602559953344),
 (('shook', 'head'), 7.6427505552465345),
 (('mr', 'mckee'), 7.6329161951419575),
 (('living', 'room'), 7.6290011451385755),
 (('half', 'hour'), 7.601889299521332),
 (('mr', 'sloane'), 7.404097504646076),
 (('never', 'loved'), 7.249587555590452),
 (('mr', 'carraway'), 7.211452426703682),
 (('five', 'year'), 7.195139771568076),
 (('four', 'clock'), 7.193004027224086),
 (('man', 'named'), 7.153663135591918),
 (('green', 'light')

# Next word probability

In this section, we shall use probability to predict the next word in a sequence of text, given a bigram or a trigram.
We can now calculate the probability of a word occurring given the two words that come before it.
By using Maximum Likelihood Estimation (MLE), write a function called calc_ngram_prob that takes in a trigram, bigram_collection, trigram_collection and returns the probability of that trigram appearing. Use the formula:
\begin{equation}
p(w_n|w_{n−2}w_{n−1}) = \frac{C(w_{n−2}w_{n−1}w_n)}{C(w_{n−2}w_{n−1})}
\end{equation}

where:
* $p(w_n|w_{n−2}w_{n−1})$ is the probbility of the word $w_n$ occurring after the bigram $(w_{n−2}w_{n−1})$.
* $C(w_{n−2}w_{n−1}w_n)$ is the count of the trigram $(w_{n−2}w_{n−1}w_n)$ in the corpus.
* $C(w_{n−2}w_{n−1})$ is the count of the bigram $(w_{n−2}w_{n−1})$ in the corpus.

This should give the likelihood of $w_n$ following $w_{n-2}$ and $w_{n-1}$.

To ensure numerical stability, we'll operate with logarithmic probabilities. Encapsulate the division operation within a math.log function call to transform it into a logarithmic probability. Additionally, it's essential to address scenarios where the bigram or trigram is absent from the provided collection. In such cases, return negative infinity to indicate out-of-vocabulary n-grams.

Try out the function on a couple of trigrams on your text of preference.


In [ ]:
def calc_ngram_prob(trigram, bigram_collection, trigram_collection):
    # Extract the components of the trigram
    w1, w2, w3 = trigram
    bigram_count = 0
    trigram_count = 0

    ## -- Check if the bigram and trigram exist in the collections -- ##

    # Check that the bigram exists
    bigram_found = False
    for bigram, freq in bigram_collection:
        if bigram == (w1, w2):
            bigram_count = freq
            bigram_found = True
            break

    # If the bigram is not found
    if not bigram_found:
        return float("-inf")  # Return negative infinity for out-of-vocabulary bigrams

    # Check that the trigram exists
    trigram_found = False
    for trigram_tuple, freq in trigram_collection:
        if trigram_tuple == (w1, w2, w3):
            trigram_count = freq
            trigram_found = True
            break

    # If the trigram does not exist
    if not trigram_found:
        return float("-inf")  # Return negative infinity for out-of-vocabulary trigrams

    # Calculate the probability
    if bigram_count == 0:
        return float("-inf")  # Return negative infinity for out-of-vocabulary bigrams
    else:
        log_prob = math.log(trigram_count / bigram_count) if trigram_count > 0 else float("-inf")
        return log_prob

In [ ]:
# The next word probability using the trigram ('west', 'egg', 'village') from the Great Gatsby
next_word_prob = calc_ngram_prob(('west', 'egg', 'village'), gatsby_bigrams, gatsby_trigrams)
print(next_word_prob)

-1.5686159179138452


In [ ]:
# Use Adventures of Huckleberry Finn/ The Adventures of Sherlock Holmes to calculate ngram probability

# Your code

In [ ]:
# Get the PMI of Adventures of Huckleberry Finn/The Adventures of Sherlock Holmes

# Your code here

## Next word Prediction
In the code cells below, we shall predict the next possible words by using probability. To do this, we create two functions:
* The first function provides a list of the next possible words.
* The second function provides he most likely next word.

In [ ]:
def get_possible_next_words(sequence, bigram_collection, trigram_collection):
    # Tokenize the input sequence
    tokens = sequence.lower().split()

    # Extract the last two words as the context for trigrams
    context = tokens[-2:]

    # Initialize a dictionary to store possible next words and their probabilities
    possible_next_words = {}

    # Iterate through the trigram collection
    for trigram, freq in trigram_collection:
        # Check if the first two words of the trigram match the context
        if trigram[:2] == tuple(context):
            next_word = trigram[2]  # Extract the next word from the trigram
            # Calculate the probability of the next word given the context
            prob = calc_ngram_prob(trigram, bigram_collection, trigram_collection)
            # Update the dictionary with the next word and its probability
            possible_next_words[next_word] = prob

    # Sort the dictionary by probabilities in descending order
    sorted_next_words = sorted(possible_next_words.items(), key=lambda x: x[1], reverse=True)

    return sorted_next_words

def predict_next_word(sequence, bigram_collection, trigram_collection):
    # Get possible next words along with their probabilities
    possible_next_words = get_possible_next_words(sequence, bigram_collection, trigram_collection)

    # If no possible next words are found, return None
    if not possible_next_words:
        return None

    # Return the most likely next word (the first element of the sorted list)
    return possible_next_words[0][0]


In [ ]:
# Get the possible next words and most likely next word for a text sequence
text_seq = "old sport"
n = 10 # set a limit for the number of possible next words

# possible next words
possible_next_word = get_possible_next_words(text_seq, gatsby_bigrams, gatsby_trigrams)
print(f"possible next words for \'{text_seq}': \n{possible_next_word[:n]}")

# most likely next word
most_likely_next_word = predict_next_word(text_seq, gatsby_bigrams, gatsby_trigrams)
print(f"\nmost likely next word for \'{text_seq}': \n{most_likely_next_word} ")

possible next words for 'old sport': 
[('gatsby', -2.70805020110221), ('near', -3.8066624897703196), ('afraid', -3.8066624897703196), ('urge', -3.8066624897703196), ('familiar', -3.8066624897703196), ('good', -3.8066624897703196), ('lunch', -3.8066624897703196), ('jump', -3.8066624897703196), ('break', -3.8066624897703196), ('great', -3.8066624897703196)]

most likely next word for 'old sport': 
gatsby 


In [ ]:
# Get the possible next words and most likely next word of a text sequence of your choice

# You can use any text sequence from any corpus

# Your code here


# Word Embeddings (Optional)
Word embeddings are numerical representations of words in a lower-dimensional space that capture their semantic and syntactic information. Word embeddings act as the digital DNA for words in natural language processing (NLP). Essentially, they transform words into numerical vectors (arrays of numbers) that machine learning algorithms can process.

Imagine these vectors as numeric fingerprints for each word. For instance, the word "apple" might be represented by a vector like [0.1, -0.3, 0.6].They help machines grasp the meaning and nuances behind each word.

For example, if "apple" is close to "fruit" in this numerical space but far from "car," the machine understands that an apple is more related to fruits than to vehicles.

Moreover, word embeddings encode relationships between words. Words that frequently appear together in the same context will have similar or 'closer' vectors.

A classic example is, in the numerical space, the vectors for "king" and "queen" might be closer to each other than those for "king" and "apple." This is because the algorithm has learned from vast amounts of text that "king" and "queen" often appear in similar contexts, such as discussions about royalty, while "king" and "apple" do not.

![word_embeddings_space](https://www.freecodecamp.org/news/content/images/size/w1600/2023/09/Screenshot-2023-09-24-at-5.43.04-PM.png)


## Vectorization

ML algorithms do not understand text, but they do understand math and in turn vectors and matrices.

### Count Vectorization
Vectorization of the text based on the counts of the words. Frequency of unique words is got and represented as a matrix, each row would represent a document (or a chunk of text, if the text is from one book). Each column would represent a unique word in the matrix. CountVectorizer uses a bag of words approach to create a vector representation.

### Term Frequency, Inverse Document Frequency (TF-IDF)
Weighs each term in a document by how unique it is to the given document it is contained in, this can allow us to create a summary of the document's contents using a few key words. In simple terms, it answers the question, "which words are the most important in a document?"
The formula for term frequency is:
\begin{equation}
TF(t) = \frac{N}{T}
\end{equation}
Where:
* N is the Number of times t appears in a document
* T is the total number of terms in the document

The formula for inverse document frequency is:

\begin{equation}
IDF(t) = log_e \frac{N} {N_{with-t}}
\end{equation}

where:
* t is a term (word)
* N is the total number of documents
* $N_{with-t}$ is the number of documents with t in it

In the code cells below, we shall use sklearn's [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) and [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

In [ ]:
# Create a corpus of documents from some texts based on the great gatsby - this is totally random feel free to use any corpus of your choice
# The corpus is sampled in this way to allow for easy demonstration
# each item in the below list is considered as a document
corpus = [g_gatsby_cleaned1[:200],
          g_gatsby_cleaned1[200:501],
          g_gatsby_cleaned1[502:808],
          g_gatsby_cleaned1[809:1004],
          g_gatsby_cleaned1[998:1204],
          g_gatsby_cleaned1[1205:1405],
          g_gatsby_cleaned1[1406:1608]]

print(corpus)

['young vulnerable year father give advice turn mind feel like criticize tell remember people world advantage unusually communicative reserve way understand mean great deal consequence inclined reserve ', 'judgement habit open curious nature victim veteran bore abnormal mind quick detect attach quality appear normal person come college unjustly accuse politician privy secret grief wild unknown man confidence unsought frequently feign sleep preoccupation hostile levity realize unmistakable sign intimate', 'revelation quiver horizon intimate revelation young man term express usually plagiaristic mar obvious suppression reserve judgement matter infinite hope little afraid miss forget father snobbishly suggest snobbishly repeat sense fundamental decency parcel unequally birth boast way tolerance come admission', 'limit conduct found hard rock wet marsh certain point care found come east autumn feel want world uniform sort moral attention forever want riotous excursion privileged glimpse hu

In [ ]:
# Create a corpus of your choice, use the cleaned text from Adventures of Huckleberry Finn/ The Adventures of Sherlock Holmes

# Your code here


In [ ]:
# Init count vectorizer
count_vect = CountVectorizer()
tfidf_vect = TfidfVectorizer()

# Fit transform on the corpus
X_count_gatbsy = count_vect.fit_transform(corpus) # count vectorizer
X_tfidf_gatsby = tfidf_vect.fit_transform(corpus) # Tfidfvectorizer

# Get the feature names (tokens)
gatsby_feat_names_count = count_vect.get_feature_names_out()
gatsby_feat_names_tfidf = tfidf_vect.get_feature_names_out()

# Print the feature names
print(f"Great Gatsby CountVectorizer feature names:\n {gatsby_feat_names_count}")
print(f"\nGreat Gatsby TfidfVectorizer feature names:\n {gatsby_feat_names_tfidf}")

Great Gatsby CountVectorizer feature names:
 ['abnormal' 'abortive' 'accuse' 'actual' 'admission' 'advantage' 'advice'
 'afraid' 'appear' 'attach' 'attention' 'autumn' 'away' 'birth' 'boast'
 'book' 'bore' 'buccleuch' 'care' 'carraway' 'certain' 'city' 'clan'
 'close' 'college' 'come' 'communicative' 'conduct' 'confidence'
 'consequence' 'creative' 'criticize' 'curious' 'deal' 'decency' 'descend'
 'detect' 'dignified' 'dream' 'duke' 'dust' 'earthquake' 'east' 'elation'
 'end' 'excursion' 'exempt' 'express' 'extraordinary' 'family' 'father'
 'feel' 'feign' 'find' 'flabby' 'float' 'forever' 'forget' 'foul' 'found'
 'frequently' 'fundamental' 'gatsby' 'generation' 'gesture' 'gift' 'give'
 'glimpse' 'gorgeous' 'great' 'grief' 'habit' 'hard' 'heart' 'heighten'
 'hope' 'horizon' 'hostile' 'housand' 'human' 'impressionability'
 'inclined' 'infinite' 'interest' 'intimate' 'intricate' 'judgement'
 'levity' 'life' 'like' 'likely' 'limit' 'little' 'machine' 'man' 'mar'
 'marsh' 'matter' 'mean' 'm

In [ ]:
# Fit the vectorizers of your corpus obtain the feature names and the matrices

# Your code here

In [ ]:
# Print the document-term matrices
print("CountVectorizer document-term matrix:")
print(X_count_gatbsy.toarray())

print("\nTfidfVectorizer document-term matrix:")
print(X_tfidf_gatsby.toarray())

CountVectorizer document-term matrix:
[[0 0 0 ... 1 1 1]
 [1 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
TfidfVectorizer document-term matrix:
[[0.         0.         0.         ... 0.16615829 0.20017    0.16615829]
 [0.16299733 0.         0.16299733 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.13173276]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.19331996 0.         ... 0.         0.         0.        ]]


What is the noticeable difference between the two vectorization techniques?

In [ ]:
# Get the TF-IDF scores for each document
for i in range(len(corpus)):
    tfidf_values = X_tfidf_gatsby[i].toarray().flatten()  # TF-IDF values for the i-th document
    max_tfidf_index = tfidf_values.argmax()  # Index of the term with the highest TF-IDF score
    max_tfidf_term = gatsby_feat_names_tfidf[max_tfidf_index]  # Term with the highest TF-IDF score
    max_tfidf_score = tfidf_values[max_tfidf_index]  # Highest TF-IDF score
    print(f"Document {i+1}: The term '{max_tfidf_term}' has the highest TF-IDF score of {max_tfidf_score}")

Document 1: The term 'reserve' has the highest TF-IDF score of 0.3323165859445813
Document 2: The term 'abnormal' has the highest TF-IDF score of 0.16299732751649293
Document 3: The term 'revelation' has the highest TF-IDF score of 0.31739550733042693
Document 4: The term 'want' has the highest TF-IDF score of 0.358395771054583
Document 5: The term 'gatsby' has the highest TF-IDF score of 0.28916926679488314
Document 6: The term 'find' has the highest TF-IDF score of 0.3843251299600989
Document 7: The term 'abortive' has the highest TF-IDF score of 0.19331996379394867


In [ ]:
# get the TF-IDF scores for your corpus of choice

# Your code here